# Imports

In [1]:
import random
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from aif360.datasets import GermanDataset
from aif360.algorithms.preprocessing import Reweighing, OptimPreproc, LFR, DisparateImpactRemover
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools
from aif360.algorithms.preprocessing.optim_preproc_helpers.distortion_functions  import get_distortion_german, get_distortion_adult
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_german, load_preproc_data_adult
from copy import deepcopy
import numpy as np
from test import test_LFR

from FairBoost.main import FairBoost, Bootstrap_type
from FairBoost import wrappers

2022-03-26 16:31:11.714788: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-26 16:31:11.714819: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
np.random.seed(42)

# Debug

In [3]:
# !pip install ipdb
import  ipdb

In [4]:
%pdb off

Automatic pdb calling has been turned OFF


# Code

In [5]:
dataset_orig = load_preproc_data_adult()
dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

In [6]:
optim_options = {
    "distortion_fun": get_distortion_adult,
    "epsilon": 0.05,
    "clist": [0.99, 1.99, 2.99],
    "dlist": [.1, 0.05, 0] 
    }   


privileged_groups = [{'sex': 1.0}]
unprivileged_groups = [{'sex': 0.0}]
pp2 = OptimPreproc(OptTools, optim_options,
                  unprivileged_groups = unprivileged_groups,
                  privileged_groups = privileged_groups)
pp2 = wrappers.OptimPreproc(pp2)

Privileged and unprivileged groups specified will not be used. The protected attributes are directly specified in the data preprocessing function. The current implementation automatically adjusts for discrimination across all groups. This can be changed by changing the optimization code.


In [7]:

pp1 = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
pp1 = wrappers.Reweighing(pp1)


In [8]:
pp3 = LFR(unprivileged_groups=unprivileged_groups,
         privileged_groups=privileged_groups,
         k=10, Ax=0.1, Ay=1.0, Az=2.0,
         verbose=1
        )
pp3 = wrappers.LFR(pp3)

In [9]:
pp4 = DisparateImpactRemover(repair_level=.5)
pp4 = wrappers.DIR(pp4)

In [10]:
pp = [pp1,pp2,pp4]

In [11]:
model = LogisticRegression()

ens = FairBoost(model, pp, bootstrap_type=Bootstrap_type.CUSTOM)
ens = ens.fit(dataset_orig_train)
y_pred = ens.predict(dataset_orig_test)
accuracy_score(y_pred, dataset_orig_test.labels)

Transforming data set with:
	-Reweighing
	-OptimPreproc
	-DisparateImpactRemover
Transforming data set with:
	-Reweighing
	-OptimPreproc
	-DisparateImpactRemover


0.7943083327646215

In [12]:
# model = LogisticRegression()
# model = model.fit(dataset_orig_train.features, dataset_orig_train.labels.ravel())
# y_pred = model.predict(dataset_orig_test.features)
# accuracy_score(y_pred, dataset_orig_test.labels)

DIR

In [13]:
# pp_ = DisparateImpactRemover(repair_level=1)
# dataset_orig_train_m = pp_.fit_transform(dataset_orig_train)
# y = deepcopy(dataset_orig_test.labels)
# dataset_orig_test_m = pp_.fit_transform(dataset_orig_test)
# model = LogisticRegression()
# model = model.fit(dataset_orig_train_m.features, dataset_orig_train_m.labels.ravel())
# y_pred = model.predict(dataset_orig_test_m.features)
# accuracy_score(y_pred, y)

OPTIM


LFR

In [14]:
# test_LFR(dataset_orig_train, dataset_orig_test)